In [2]:
import numpy as np

# Constants
n_iterations = 100  # number of iterations
n_device = [50, 100, 150, 200, 250, 300]  # number of devices
n_mecs = 10  # number of MECs
mec_capacity = 4  # MEC computing capacity (GHz)
transmission_power = 0.5  # Transmission power (W)

# Voltages (assumed constants)
mec_voltage = 1.0  # Assumed voltage for MEC
device_voltage = 1.0  # Assumed voltage for device

# Task data (Mbit)
task_data = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45])

# Transmission rate (kb/s)
transmission_rate = np.array([500, 1000, 1500, 2000, 2500, 3000]) * 1e-3  # Convert to Mbit/s

def fitness(position, device_capacity, td, tr):
    total_transmission_time = 0
    total_computation_time = 0
    total_energy_consumption = 0

    # Calculate the transmission time and the computation time
    transmission_time = td / tr
    total_transmission_time += transmission_time

    computation_time = td / (position * mec_capacity + (1 - position) * device_capacity)
    total_computation_time += computation_time

    # Calculate energy consumption
    energy_computation_mec = position * mec_capacity * mec_voltage**2  # energy consumed at MEC
    energy_computation_device = (1 - position) * device_capacity * device_voltage**2  # energy consumed at device
    energy_transmission = transmission_power * transmission_time  # energy consumed during transmission

    total_energy_consumption += energy_transmission + energy_computation_mec + energy_computation_device

    # Calculate the mean completion time (total time divided by number of tasks)
    mean_completion_time = (total_transmission_time + total_computation_time)
    # Total cost is sum of all transmission and computation times
    cost = total_transmission_time + total_computation_time

    return cost, total_energy_consumption, mean_completion_time

# Iterate over all device settings
for n in n_device:
    # Set the number of particles
    n_particles = n * n_mecs  # each particle represents a task offloading decision for a device to a MEC

    # Device computing capacity (GHz)
    device_capacity = np.random.uniform(0.5, 1, n_particles)  # generate device capacity for each particle

    # Task data and transmission rate for each particle
    task_data_per_particle = np.random.choice(task_data, n_particles)
    transmission_rate_per_particle = np.random.choice(transmission_rate, n_particles)

    # Initialize the particles' positions
    particles = np.random.uniform(low=0, high=1, size=(n_particles))  # range changed to 0 to 1

    # Initialize the personal best positions and the global best position
    pbest_positions = particles
    pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
    gbest_fitness_value = float('inf')
    gbest_position = 0
    gbest_energy_consumption = float('inf')
    gbest_mean_completion_time = float('inf')

    # Start iterations
    for i in range(n_iterations):
        for j in range(n_particles):
            fitness_cadidate, energy_consumption, mean_completion_time = fitness(particles[j], device_capacity[j], task_data_per_particle[j], transmission_rate_per_particle[j])  # pass device capacity as an argument
            if(pbest_fitness_value[j] > fitness_cadidate):
                pbest_fitness_value[j] = fitness_cadidate
                pbest_positions[j] = particles[j]

            if(gbest_fitness_value > fitness_cadidate):
                gbest_fitness_value = fitness_cadidate
                gbest_position = particles[j]
                gbest_energy_consumption = energy_consumption
                gbest_mean_completion_time = mean_completion_time

        # update position based on quantum behaviour and ensure it's in [0, 1]
        particles = pbest_positions + 0.5 * np.random.rand(n_particles) * (gbest_position - pbest_positions)
        particles = np.clip(particles, 0, 1)  # keep particles within [0, 1]

    print(f"For {n} devices, the best position is {gbest_position}, with energy consumption {gbest_energy_consumption}, "
          f"and mean completion time {gbest_mean_completion_time} in iteration number {n_iterations}")


For 50 devices, the best position is 0.9392775456904482, with energy consumption 4.647724050088221, and mean completion time 2.977492004521352 in iteration number 100
For 100 devices, the best position is 0.966127097385158, with energy consumption 4.731541280556855, and mean completion time 2.949307323012674 in iteration number 100
For 150 devices, the best position is 0.918631827667126, with energy consumption 4.578500135212433, and mean completion time 3.001720688156791 in iteration number 100
For 200 devices, the best position is 0.9678269803707206, with energy consumption 4.736524710190369, and mean completion time 2.947669701682118 in iteration number 100
For 250 devices, the best position is 0.9346968183485083, with energy consumption 4.637128853165626, and mean completion time 2.981143213560662 in iteration number 100
For 300 devices, the best position is 0.9947619818566532, with energy consumption 4.817580486935001, and mean completion time 2.9216088947059378 in iteration numbe